In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Convert labels to one-hot vectors

# Convert classes to indicator vectors
def one_hot(values,n_values=10):
    n_v = np.maximum(n_values,np.max(values) + 1)
    oh=np.eye(n_v)[values]
    return oh


/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/amit/anaconda2/envs/myenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
print(tf.__version__)

1.9.0


# Get Mnist data and split into train validation and test

In [3]:
def get_mnist():

    labels=np.float32(np.load('/project/cmsc25025/mnist/MNIST_labels.npy'))
    data=np.float64(np.load('/project/cmsc25025/mnist/MNIST.npy'))
    print(data.shape)
    data=np.float32(data)/255.
    train_dat=data[0:50000]
    train_labels=one_hot(np.int32(labels[0:50000]))
    val_dat=data[50000:60000]
    val_labels=one_hot(np.int32(labels[50000:60000]))
    test_dat=data[60000:70000]
    test_labels=one_hot(np.int32(labels[60000:70000]))
    
    return (train_dat, train_labels), (val_dat, val_labels), (test_dat, test_labels)

In [4]:
HH=get_mnist()

(70000, 784)


In [5]:
# Convolution layer 

In [6]:
def conv_layer(input,filter_size=[3,3],num_features=[1]):

    # Get number of input features from input and add to shape of new layer
    shape=filter_size+[input.get_shape().as_list()[-1],num_features]
    W = tf.get_variable('W',shape=shape) # Default initialization is Glorot (the one explained in the slides)
    #b = tf.get_variable('b',shape=[num_features],initializer=tf.zeros_initializer) 
    conv = tf.nn.conv2d(input, W, strides=[1, 1, 1, 1], padding='SAME')
    
    return(conv)

In [7]:
def fully_connected_layer(input,num_features):
    # Make sure input is flattened.
    flat_dim=np.int32(np.array(input.get_shape().as_list())[1:].prod())
    input_flattened = tf.reshape(input, shape=[-1,flat_dim])
    shape=[flat_dim,num_features]
    W_fc = tf.get_variable('W',shape=shape) 
    #b_fc = tf.get_variable('b',shape=[num_features],initializer=tf.zeros_initializer)
    fc = tf.matmul(input_flattened, W_fc) # + b_fc
    return(fc)

In [8]:
from keras import backend as K
from keras.layers.convolutional import UpSampling2D
 
def MaxPoolingandMask(inputs, pool_size, strides,
                          padding='SAME'):

        pooled = tf.nn.max_pool(inputs, ksize=pool_size, strides=strides, padding=padding)
        upsampled = UpSampling2D(size=strides[1:3])(pooled)
        indexMask = K.tf.equal(inputs, upsampled)
        assert indexMask.get_shape().as_list() == inputs.get_shape().as_list()
        return pooled,indexMask
     
#def get_output_shape_for(self, input_shape):
#        return input_shape
 
 
def unpooling(x,mask,strides):
    '''
    do unpooling with indices, move this to separate layer if it works
    1. do naive upsampling (repeat elements)
    2. keep only values in mask (stored indices) and set the rest to zeros
    '''
    on_success = UpSampling2D(size=strides)(x)
    on_fail = K.zeros_like(on_success)
    return K.tf.where(mask, on_success, on_fail)
 
 


Using TensorFlow backend.


In [9]:
def create_small_network():

    with tf.variable_scope("conv1"):
            # Variables created here will be named "conv1/weights", "conv1/biases".
            conv1 = conv_layer(x_image, filter_size=[3, 3],num_features=16)
    with tf.variable_scope("conv1K"):
            conv1K = tf.clip_by_value(conv1,-1.,1.)
    with tf.variable_scope("pool1"):
            pool1, mask1 = MaxPoolingandMask(conv1, pool_size=[1,3,3,1], strides=[1,2,2,1])
    print('pool1','mask1',pool1.shape,mask1.shape)
    with tf.variable_scope("fc"):
            fc = fully_connected_layer(pool1, num_features=10)
    with tf.variable_scope('cross_entropy_loss'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=fc),name="LOSS")

#     with tf.variable_scope("conv1_a"):
#             # Variables created here will be named "conv1/weights", "conv1/biases".
#             conv1_a = conv_layer(x_image, filter_size=[3, 3],num_features=4)
#     with tf.variable_scope("fc_a"):
#             fc_a = fully_connected_layer(conv1_a, num_features=10)
#     with tf.variable_scope('cross_entropy_loss_a'):
#         cross_entropy_a = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=fc_a),name="LOSSa")

    # Accuracy computation
    with tf.variable_scope('helpers'):
        correct_prediction = tf.equal(tf.argmax(fc, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="ACC")
    # We return the final functions (they contain all the information about the graph of the network)
    return cross_entropy, accuracy, conv1, conv1K, fc, pool1, mask1

In [10]:
# Get loss and accuracy on a data set with output from final layer fc.

In [11]:
# Function to get loss and accuracy from only one run of the feature extraction network
from scipy.special import logsumexp

def get_stats(data,labels):
    t1=time.time()
    lo=0.
    acc=0.
    delta=1000
    rr=np.arange(0,data.shape[0],delta)
    for i in rr:
        fc_out=fc.eval(feed_dict={x: data[i:i+delta], y_:labels[i:i+delta]})
        log_sf=logsumexp(fc_out,axis=1).reshape((fc_out.shape[0],1))-fc_out
        lo+=np.mean(np.sum(labels[i:i+delta]*log_sf, axis=1))
        acc += np.mean(np.equal(np.argmax(fc_out, axis=1),np.argmax(labels[i:i+delta], axis=1)))
    acc=acc/np.float32(len(rr))
    lo=lo/np.float32(len(rr))
    print('get stats time',time.time()-t1)
    # We return the final functions (they contain all the information about the graph of the network)
    return lo, acc

# Run one epoch

In [20]:
def back_prop(): #(VS,cross_entropy,conv,fc):
    #convx, fcx = sess.run([conv,fc],feed_dict={x: batch[0], y_: batch[1]})
    # Get gradient of loss with respect to final output layer using tf gradient
    # The rest will be explicit backprop
    gradx=tf.gradients(cross_entropy,fc)
    #gradx=sess.run(GRADx,feed_dict={x: batch[0], y_: batch[1]})
    #print('gradx',gradx)
    # Flatten convolutional output layer
    poolf=tf.contrib.layers.flatten(pool)
    # Gradient of weights of dense layer
    gradfcW=tf.matmul(tf.transpose(poolf),gradx[0])
    # Propagated error to conv layer.
    gradfcx_pool=tf.matmul(gradx[0],tf.transpose(VS[1]))
    # Reshape propagated error to spatial dims.
    gradfcx_pool=tf.reshape(gradfcx_pool,[-1]+(pool.shape.as_list())[1:])
    gradfcx=unpooling(gradfcx_pool,mask,[2,2])
    #print('gradfcx',gradfcx.shape)
    # Get gradient of conv layer W's
    w_shape=VS[0].shape
    strides=[1,1,1,1]
    out_backprop=tf.reshape(gradfcx,[-1]+(conv.shape.as_list())[1:])
    on_fail = K.zeros_like(out_backprop)
    out_backpropF=K.tf.where(tf.equal(tf.abs(convK),1.),out_backprop,on_fail)
    gradconvW=tf.nn.conv2d_backprop_filter(input=x_image,filter_sizes=w_shape,\
                                                     out_backprop=out_backpropF,\
                                                     strides=strides,\
                                                     padding='SAME')
    #print('gradconvW',gradconvW)
    #print('gradfcW',gradfcW)
    return gradconvW, gradfcW

In [21]:

# Run the iterations of one epoch
def run_epoch(train,val,ii,batch_size,cross_entropy,dWs,step_size):
        t1=time.time()
        # Randomly shuffle the training data
        np.random.shuffle(ii)
        tr=train[0][ii]
        y=train[1][ii]
        lo=0.
        acc=0.
        #VS=tf.trainable_variables()
        # Run disjoint batches on shuffled data
        for j in np.arange(0,len(y),batch_size):
            if (np.mod(j,5000)==0):
                print('Batch',j/batch_size)
            batch=(tr[j:j+batch_size],y[j:j+batch_size])
            grad=sess.run(dWs,feed_dict={x: batch[0], y_: batch[1]})
            
            for gr, vs in zip(grad,VS):
              sess.run(vs.assign(vs - step_size*gr))
        print('Epoch time',time.time()-t1)

# Get the data and run the training. Save the model and test at the end

In [22]:
def get_data(data_set):
    if (data_set=="cifar"):
        return(get_cifar())
    elif (data_set=="mnist"):
        return(get_mnist())
    elif (data_set=="mnist_transform"):
        return(get_mnist_trans())

In [23]:
# Run the training

import time
batch_size=100
step_size=.1
num_epochs=10
num_train=50000
minimizer="SGD"
data_set="mnist"
model_name="model"
keep_prob=.5
dim=28
nchannels=1
if (data_set=="cifar"):
    dim=32
    nchannels=3

    
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, dim*dim*nchannels],name="x")
x_image = tf.reshape(x, [-1, dim, dim, nchannels])
# Dimensions of x_image: [Batch size, Column size, Row size, Number of incoming channels]
# The number of incoming channels, for example, will be 3 if the image is color: RGB (red, green, blue)
# We will slide filter over this 2d picture with conv2d function.
y_ = tf.placeholder(tf.float32, shape=[None,10],name="y")
# Allows you to control the time step during the iterations
lr_ = tf.placeholder(tf.float32, shape=[],name="learning_rate")
keep_prob_=tf.placeholder(tf.float32, shape=[],name="keep_prob")

with tf.Session() as sess:
    train,val,test=get_data(data_set=data_set)
    
    # Create the network architecture with the above placeholdes as the inputs.
    cross_entropy, accuracy, conv, convK, fc, pool, mask =create_small_network()
    #train_step = tf.train.GradientDescentOptimizer(learning_rate=lr_).minimize(cross_entropy)
    # Initialize variables
    sess.run(tf.global_variables_initializer())

    # Show trainable variables
    VS=tf.trainable_variables()
    for v in VS:
        print(v.name,v.get_shape().as_list(),np.std(v.eval()))
    dWs=back_prop() #(VS,cross_entropy,conv,fc)
    ii=np.arange(0,num_train,1) #len(train_data),1)

    # Run epochs
    for i in range(num_epochs):  # number of epochs
        run_epoch(train,val,ii,batch_size,cross_entropy,dWs,step_size)
        
        if (np.mod(i,2)==0):
            lo,ac = get_stats(train[0][0:num_train],train[1][0:num_train])
            print('Epoch',i,'Train loss, accuracy',lo,ac)
            vlo,vac = get_stats(val[0],val[1])
            print('EPoch',i,'Validation loss, accuracy',vlo,vac)
            # Test set accuracy
 
    print('test accuracy %g' % accuracy.eval(feed_dict={x: test[0], y_:test[1]}))
    
    # Save model
    #tf.add_to_collection("optimizer", train_step)
    saver = tf.train.Saver()
    save_path = saver.save(sess, "tmp/"+model_name)
    print("Model saved in path: %s" % save_path)


(70000, 784)
pool1 mask1 (?, 14, 14, 16) (?, 28, 28, 16)
conv1/W:0 [3, 3, 1, 16] 0.11082865
fc/W:0 [3136, 10] 0.025171598
Batch 0.0
Batch 50.0
Batch 100.0
Batch 150.0
Batch 200.0
Batch 250.0
Batch 300.0
Batch 350.0
Batch 400.0
Batch 450.0
Epoch time 66.12982511520386
get stats time 3.3394076824188232
Epoch 0 Train loss, accuracy 0.4575508047032357 0.8913399999999999
get stats time 0.8108248710632324
EPoch 0 Validation loss, accuracy 0.4319435055494309 0.9037
Batch 0.0
Batch 50.0
Batch 100.0
Batch 150.0
Batch 200.0
Batch 250.0


KeyboardInterrupt: 

# Reload the model that was saved and continue training

In [ ]:
# Reloading an existing model.

tf.reset_default_graph()
batch_size=500
step_size=.001
num_epochs=4
num_train=10000
data_set="cifar"
model_name="model"
Train=True
dim=28
nchannels=1
if (data_set=="cifar"):
    dim=32
    nchannels=3
    
with tf.Session() as sess:
    # Get data
    train, val, test=get_data(data_set=data_set)
    # Load model info
    saver = tf.train.import_meta_graph('tmp/'+model_name+'.meta')
    saver.restore(sess,'tmp/'+model_name) 
    graph = tf.get_default_graph()
    # Setup the placeholders from the stored model.
    x = graph.get_tensor_by_name('x:0')
    y_= graph.get_tensor_by_name('y:0')
    lr_ = graph.get_tensor_by_name('learning_rate:0')
    keep_prob_ = graph.get_tensor_by_name('keep_prob:0')
    accuracy=graph.get_tensor_by_name('helpers/ACC:0')
    cross_entropy=graph.get_tensor_by_name('cross_entropy_loss/LOSS:0')
    fc=graph.get_tensor_by_name('OUT:0')
    # Get the minimization operation from the stored model
    if (Train):
        train_step_new = tf.get_collection("optimizer")[0]
        # Confirm training accuracy of current model before additional training
        acc=accuracy.eval(feed_dict={x: train[0][0:num_train], y_:train[1][0:num_train]})
        print('train acc',acc)

        ii=np.arange(0,num_train,1) 
        for i in range(num_epochs):  # Run epochs
            run_epoch(train,val,ii,batch_size,train_step_new)
            if (np.mod(i,2)==0):
                lo,ac = get_stats(train[0][0:num_train],train[1][0:num_train])
                print('Epoch',i,'Train loss, accuracy',lo,ac)
                vlo,vac = get_stats(val[0],val[1])
                print('EPoch',i,'Validation loss, accuracy',vlo,vac)
    # Test set accuracy

    print('test accuracy %g' % accuracy.eval(feed_dict={x: test[0], y_:test[1]}))
    
    tf.add_to_collection("optimizer", train_step)
    saver = tf.train.Saver()
    save_path = saver.save(sess, "tmp/"+model_name)
    print("Model saved in path: %s" % save_path)